In [1]:
import pandas as pd
import numpy as np

In [2]:
path="10000_TestSet.xlsx"
df_test = pd.read_excel(path, sheet_name = 'Sheet1')

df_test.drop(columns = '序号', inplace = True)

df_test.columns = ['text','category']

df_test['category'].replace(to_replace=1,value='不通顺',inplace=True)
df_test['category'].replace(to_replace=0,value='通顺',inplace=True)

df_test.head()

,text,category
0,村里的事情应该由村民委员会作主。,不通顺
1,"据珠海航展公司消息称,在即将举办的中国航展上,AG600飞机将再次坐阵主场亮相",不通顺
2,"都说十月怀胎很辛苦,那么临了生产后做月子是否会哭?",不通顺
3,这个坐右铭可以使你自信。,不通顺
4,为了叫同学们睡个午觉，学校调整了做息时间。,不通顺


In [3]:
labels = {'不通顺': 0, '通顺': 1}
labels

{'不通顺': 0, '通顺': 1}

In [4]:
import torch

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df_test['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 65, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

/root/miniconda3/envs/my-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, len(labels))
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [6]:
def evaluate(model, test_data):
    
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [8]:
model = BertClassifier()

for i in range(1, 11):
    print(f"sigmoid epoch{i}", end=" ")
    
    model.load_state_dict(torch.load(f'./thu_1error_sigmoid_epoches/epoch{i}.pth'))
    evaluate(model, df_test)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


sigmoid epoch1 Test Accuracy:  0.382
sigmoid epoch2 Test Accuracy:  0.380
sigmoid epoch3 

KeyboardInterrupt: 